In [1]:
import pandas as pd
import numpy as np

pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import MinMaxScaler

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout
from tensorflow.keras.utils import to_categorical 

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [3]:
import os
import librosa
import librosa.display
import glob 
import skimage

In [4]:
df = pd.read_csv("../input/urbansound8k/UrbanSound8K.csv")
df.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [5]:
dat1, sampling_rate1 = librosa.load('../input/urbansound8k/fold5/100032-3-0-0.wav')
arr = librosa.feature.melspectrogram(y=dat1, sr=sampling_rate1)
arr.shape

(128, 14)

In [6]:
feature = []
label = []

def parser(row):
    
    for i in range(8732):
        file_name = '../input/urbansound8k/fold' + str(df["fold"][i]) + '/' + df["slice_file_name"][i]
        
        X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        
        mels = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)        
        feature.append(mels)
        label.append(df["classID"][i])
    return [feature, label]

In [7]:
temp = parser(df)

In [8]:
temp = np.array(temp)
data = temp.transpose()

In [9]:
X_ = data[:, 0]
Y = data[:, 1]
print(X_.shape, Y.shape)
X = np.empty([8732, 128])

(8732,) (8732,)


In [10]:
for i in range(8732):
    X[i] = (X_[i])

In [11]:
Y = to_categorical(Y)

In [12]:
'''Final Data'''
print(X.shape)
print(Y.shape)

(8732, 128)
(8732, 10)


In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 1)

In [14]:
X_train = X_train.reshape(6549, 16, 8, 1)
X_test = X_test.reshape(2183, 16, 8, 1)

In [15]:
input_dim = (16, 8, 1)

In [16]:
model = Sequential()

In [17]:
model.add(Conv2D(64, (3, 3), padding = "same", activation = "tanh", input_shape = input_dim))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), padding = "same", activation = "tanh"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(1024, activation = "tanh"))
model.add(Dense(10, activation = "softmax"))

In [18]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [19]:
model.fit(X_train, Y_train, epochs = 90, batch_size = 50, validation_data = (X_test, Y_test))

Train on 6549 samples, validate on 2183 samples
Epoch 1/90
6549/6549 [==============================] - 5s 820us/sample - loss: 1.5571 - accuracy: 0.4665 - val_loss: 1.2848 - val_accuracy: 0.5547
Epoch 2/90
6549/6549 [==============================] - 4s 685us/sample - loss: 1.1384 - accuracy: 0.6109 - val_loss: 1.1341 - val_accuracy: 0.6129
Epoch 3/90
6549/6549 [==============================] - 4s 643us/sample - loss: 0.9784 - accuracy: 0.6651 - val_loss: 1.0240 - val_accuracy: 0.6683
Epoch 4/90
6549/6549 [==============================] - 4s 626us/sample - loss: 0.8621 - accuracy: 0.7116 - val_loss: 0.9722 - val_accuracy: 0.6995
Epoch 5/90
6549/6549 [==============================] - 4s 635us/sample - loss: 0.7696 - accuracy: 0.7403 - val_loss: 0.9928 - val_accuracy: 0.6867
Epoch 6/90
6549/6549 [==============================] - 4s 642us/sample - loss: 0.6732 - accuracy: 0.7804 - val_loss: 0.8408 - val_accuracy: 0.7329
Epoch 7/90
6549/6549 [==============================] - 4s 637us

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 16, 8, 64)         640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 4, 64)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 4, 128)         73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 2, 128)         0         
_________________________________________________________________
dropout (Dropout)            (None, 4, 2, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              1

In [21]:
predictions = model.predict(X_test)
score = model.evaluate(X_test, Y_test)
print(score)

2183/2183 [==============================] - 0s 224us/sample - loss: 0.9989 - accuracy: 0.8388
[0.9988526380012956, 0.838754]


VAL_LOSS_CNN = 0.84

In [22]:
preds = np.argmax(predictions, axis = 1)

In [23]:
result = pd.DataFrame(preds)
result.to_csv("UrbanSound8kResults.csv")